### Data merging and cleaning

In this notebook the dataset for analysis is created. The following features are collected:
 - bitcoin price (using https://www.blockchain.com/api )
 - the total number of confirmed transactions per day (using https://www.blockchain.com/api )
 - average transaction fees in USD per bitcoin transaction (downloaded manually, API does not work)
 - gold ounce price (downloaded manually)
 - oil WTI price (downloaded manually)
 - S&P 500 index (downloaded manually)
 - M2 money supply (downloaded manually)
 - google bitcoin trends search (downloaded manually)
 
The data of bitcoin transaction fees is divided into two periods - for the last three years the data is for every day and before this period the data is for every three days (this is due to the website capabilities)

In [75]:
import pandas as pd
import numpy as np
from functools import reduce
import matplotlib.pyplot as plt
import requests
import json
from datetime import datetime

#### Bitcoin price and the total number of confirmed transactions per day

In [76]:
#function for collecting data from the website
def get_data(chartName,start_date,timespan,data_name):
    API_URL = "https://api.blockchain.info/charts/"+chartName
    search = {
        "timespan": timespan,
        "start": start_date,
        "format":"json"
     }

    response = requests.get(API_URL,search)
    data=response.json()
    
    data_df=pd.DataFrame(columns=[data_name,'Date'])
    for i in data['values']:

        data_df=data_df.append({data_name:float(i['y']),'Date':i['x']},ignore_index=True)

    data_df['Date']=data_df['Date'].apply(lambda x: datetime.utcfromtimestamp(x).strftime('%Y-%m-%d'))
    data_df['Date']=pd.to_datetime(data_df['Date'],format='%Y-%m-%d')
    data_df.set_index('Date',drop=True,inplace=True)
    return data_df

In [77]:
#collecting the data
data_to_collect=['market-price','n-transactions']
start_date='2014-01-01'
time_period='3year'
btc_data_1=pd.DataFrame(columns=data_to_collect)
for i in data_to_collect[:3]:
    df_temp=get_data(i,start_date,time_period,i)
    btc_data_1[i]=df_temp.iloc[:,0]
    del df_temp

start_date='2017-01-01'
time_period='4year'
btc_data_2=pd.DataFrame(columns=data_to_collect)
for i in data_to_collect[:3]:
    df_temp=get_data(i,start_date,time_period,i)
    btc_data_2[i]=df_temp.iloc[:,0]
    del df_temp
    
btc_data=pd.concat([btc_data_1,btc_data_2],axis=0)
btc_data.rename(columns={'market-price':'BTC price [USD]'},inplace=True)

#### Transaction fees  

In [78]:
fees3=pd.read_csv("fees-usd-per-transaction3Years.csv")
fees3['Date']=pd.to_datetime(fees3['Timestamp'],format='%Y-%m-%d').dt.date.astype('datetime64')
fees3.set_index('Date',drop=True,inplace=True)
fees3.drop('Timestamp',axis=1,inplace=True)

feesAllTime=pd.read_csv("fees-usd-per-transactionAllTime.csv")
feesAllTime['Date']=pd.to_datetime(feesAllTime['Timestamp'],format='%Y-%m-%d').dt.date.astype('datetime64')
feesAllTime.set_index('Date',drop=True,inplace=True)
feesAllTime.drop('Timestamp',axis=1,inplace=True)

date_mask =feesAllTime.index < fees3.index.min()

fees=pd.concat([feesAllTime[date_mask],fees3],axis=0)
fees.rename(columns={'fees-usd-per-transaction':'fee [USD]'},inplace=True)

__Gold ounce price__

In [79]:
gold=pd.read_csv('Gold Futures Historical Data20102020maj.csv')
gold['Date']=pd.to_datetime(gold['Date'],format='%b %d, %Y')
gold.set_index('Date',drop=True,inplace=True)
gold.drop(columns=gold.iloc[:,1:],inplace=True)
gold.rename(columns={'Price':'Gold price[USD]'},inplace=True)
gold=gold.iloc[::-1,:]

gold['Gold price[USD]']=gold['Gold price[USD]'].str.replace(",","").astype(float)

__Oil WTI price__

In [80]:
oil=pd.read_csv('wti-daily_csv.csv')
oil['Date']=pd.to_datetime(oil['Date'],format='%Y-%m-%d')
oil.set_index('Date',drop=True,inplace=True)
oil.rename(columns={'Price':'Oil WTI price[USD]'},inplace=True)

__M2 money supply in the USA__

In [81]:
M2=pd.read_csv('FRB_H6.csv')
M2.drop(index=[0,1,2,3,4],columns=M2.iloc[:,1:4],inplace=True)
may20 = {'Series Description':'2020-05', 'M2; Not seasonally adjusted':0}
M2 = M2.append(may20, ignore_index=True)
M2['Series Description']=pd.to_datetime(M2['Series Description'],format='%Y-%m')
M2.rename(columns={'Series Description':'Date','M2; Not seasonally adjusted':'M2(Not seasonally adjusted)[1e+09 USD]'},inplace=True)
M2.set_index('Date',drop=True,inplace=True)
M2=M2.resample('D').ffill()
M2.drop(index=[M2.index[-1]],inplace=True)
M2['M2(Not seasonally adjusted)[1e+09 USD]']=M2['M2(Not seasonally adjusted)[1e+09 USD]'].astype(float)

__S&P500 index__

In [82]:
SP500=pd.read_csv('GSPC.csv')
SP500['Date']=pd.to_datetime(SP500['Date'],format='%Y-%m-%d')
SP500.set_index('Date',drop=True,inplace=True)
SP500.drop(columns=SP500.iloc[:,:3],inplace=True)
SP500.drop(columns=SP500.iloc[:,1:],inplace=True)
SP500.rename(columns={'Close':'SP500 close index'},inplace=True)

__Bitcoin Google search trends__

In [83]:
search_interest=pd.read_csv("multiTimeline.csv",skiprows=1)
search_interest['Month']=pd.to_datetime(search_interest['Month'],format='%Y-%m-%d')
search_interest.set_index('Month',drop=True,inplace=True)
search_interest.index.name='Date'
search_interest.rename(columns={'bitcoin: (Worldwide)':'btc search trends'},inplace=True)
search_interest=search_interest.resample('D').ffill()

__Merging datasets__

In [84]:
def merg(df1,df2,how='left',on='Date'):
    df_res=pd.merge(df1,df2,how=how,on=on)
    return df_res

dfs=[btc_data,fees,search_interest,gold,SP500,oil,M2]
dataset=reduce(merg, dfs)

In [85]:
date_mask_cut=(dataset.index < pd.to_datetime("2020-04-30")) & (dataset.index > pd.to_datetime("2014-01-01"))
dataset=dataset[date_mask_cut]

In [86]:
dataset=dataset.round(decimals=2)
dataset.to_csv('Bitcoin_dataset_updated.csv',index=True)